<a href="https://colab.research.google.com/github/AashiDutt/ADVANCED-DEPLOYMENT-SCENARIOS-WITH-TENSORFLOW/blob/master/IMDB_REVIEWS_WITH_TF_HUB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
try:
    %tensorflow_version 2.x
except:
    pass

In [6]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

print("\u2022 Using TensorFlow Version:", tf.__version__)

• Using TensorFlow Version: 2.2.0-rc3


In [0]:
# DOWNLOAD IMDB DATASET
splits = ['train[:60%]', 'train[-40%:]', 'test']

splits, info = tfds.load(name="imdb_reviews", with_info=True, split=splits, as_supervised=True)

train_data, validation_data, test_data = splits

In [8]:
num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples
num_classes = info.features['label'].num_classes

print('The Dataset has a total of:')
print('\u2022 {:,} classes'.format(num_classes))

print('\u2022 {:,} movie reviews for training'.format(num_train_examples))
print('\u2022 {:,} movie reviews for testing'.format(num_test_examples))

The Dataset has a total of:
• 2 classes
• 25,000 movie reviews for training
• 25,000 movie reviews for testing


The labels are either 0 or 1, where 0 is a negative review, and 1 is a positive review

In [0]:
class_names = ['negative', 'positive']

In [10]:
for review, label in train_data.take(1):
    review = review.numpy()
    label = label.numpy()

    print('\nMovie Review:\n\n', review)
    print('\nLabel:', class_names[label])


Movie Review:

 b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."

Label: negative


Load Word Embeddings

In this example, the input data consists of sentences. The labels to predict are either 0 or 1.

One way to represent the text is to convert sentences into word embeddings. Word embeddings, are an efficient way to represent words using dense vectors, where semantically similar words have similar vectors. We can use a pre-trained text embedding as the first layer of our model, which will have two advantages:

We don't have to worry anout text preprocessing.
We can benefit from transfer learning.
For this example we will use a model from TensorFlow Hub called google/tf2-preview/gnews-swivel-20dim/1. We'll create a hub.KerasLayer that uses the TensorFlow Hub model to embed the sentences. We can choose to fine-tune the TF hub module weights during training by setting the trainable parameter to True.

In [0]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)

In [0]:
# build pipeline
batch_size = 512

train_batches = train_data.shuffle(num_train_examples // 4).batch(batch_size).prefetch(1)
validation_batches = validation_data.batch(batch_size).prefetch(1)
test_batches = test_data.batch(batch_size)

Build the Model

In the code below we will build a Keras Sequential model with the following layers:

The first layer is a TensorFlow Hub layer. This layer uses a pre-trained SavedModel to map a sentence into its embedding vector. The model that we are using (google/tf2-preview/gnews-swivel-20dim/1) splits the sentence into tokens, embeds each token and then combines the embedding. The resulting dimensions are: (num_examples, embedding_dimension).
This fixed-length output vector is piped through a fully-connected (Dense) layer with 16 hidden units.
The last layer is densely connected with a single output node. Using the sigmoid activation function, this value is a float between 0 and 1, representing a probability, or confidence level.

In [0]:
model = tf.keras.Sequential([
        hub_layer,
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')])

In [14]:
# train the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_batches,
                    epochs=20,
                    validation_data=validation_batches)

Epoch 1/20
30/30 [==============================] - 3s 112ms/step - loss: 0.7652 - accuracy: 0.5485 - val_loss: 0.6670 - val_accuracy: 0.6038
Epoch 2/20
30/30 [==============================] - 3s 106ms/step - loss: 0.6332 - accuracy: 0.6428 - val_loss: 0.6143 - val_accuracy: 0.6654
Epoch 3/20
30/30 [==============================] - 3s 107ms/step - loss: 0.5938 - accuracy: 0.6875 - val_loss: 0.5820 - val_accuracy: 0.6988
Epoch 4/20
30/30 [==============================] - 3s 103ms/step - loss: 0.5591 - accuracy: 0.7217 - val_loss: 0.5504 - val_accuracy: 0.7311
Epoch 5/20
30/30 [==============================] - 3s 106ms/step - loss: 0.5229 - accuracy: 0.7561 - val_loss: 0.5208 - val_accuracy: 0.7577
Epoch 6/20
30/30 [==============================] - 3s 106ms/step - loss: 0.4887 - accuracy: 0.7793 - val_loss: 0.4926 - val_accuracy: 0.7799
Epoch 7/20
30/30 [==============================] - 3s 108ms/step - loss: 0.4571 - accuracy: 0.7996 - val_loss: 0.4657 - val_accuracy: 0.7946
Epoch 

In [15]:
# evaluating model
eval_results = model.evaluate(test_batches, verbose=0)

for metric, value in zip(model.metrics_names, eval_results):
    print(metric + ': {:.3}'.format(value))

loss: 0.313
accuracy: 0.867
